In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
import PIL
import time
import glob
from tqdm import tqdm

from tensorflow.keras import layers
from IPython import display

In [ ]:
# Carregando os dados do MNIST
(train_imgs, train_labels), _ = tf.keras.datasets.mnist.load_data()

In [ ]:
# normalizando para -1 e 1
train_imgs = np.expand_dims(train_imgs, axis=-1).astype(np.float32)
train_imgs = (train_imgs - 127.5) / 127.5

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
LATENT_DIM = 100
N_IMG_TO_GENERATE = 8
IMAGE_SIZE = 28, 28
CHANNELS = 1,
IMAGE_SHAPE = IMAGE_SIZE + CHANNELS 

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(train_imgs).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
GENERATOR = tf.keras.Sequential([
    layers.Dense(7*7*256, use_bias=False, input_shape=(LATENT_DIM,)),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
    
    layers.Reshape((7, 7, 256)),
    
    layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
    
    layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
    
    layers.Conv2DTranspose(CHANNELS, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'),
])

In [ ]:
# Criando uma imagem com o gerador (não treinado)
noise = tf.random.normal([1, LATENT_DIM])
generated_image = GENERATOR(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
DISCRIMINATOR = tf.keras.Sequential([
    layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=IMAGE_SHAPE),
    layers.LeakyReLU(),
    layers.Dropout(.2),
    
    layers.Conv2D(128, (5, 5), strides=(1, 1), padding='same'),
    layers.LeakyReLU(),
    layers.Dropout(.2),
    
    layers.Flatten(),
    layers.Dense(1)
])

In [ ]:
decision = DISCRIMINATOR(generated_image)
print (decision)

In [ ]:
# definindo as funções de erro
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


def generator_loss(fake_output):
    loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return loss

In [ ]:
# definindo os otimizadores
discriminator_optim = tf.keras.optimizers.Adam(1e-4)
generator_optim = tf.keras.optimizers.Adam(1e-4)

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
    
    # DISCRIMINATOR TRAIN
    with tf.GradientTape() as disc_tape:
        fake_img = GENERATOR(noise, training=False)
        
        fake_out = DISCRIMINATOR(fake_img, training=True)
        real_out = DISCRIMINATOR(images, training=True)
        
        disc_loss = discriminator_loss(real_out, fake_out)
        
    disc_grads = disc_tape.gradient(disc_loss, DISCRIMINATOR.trainable_weights)
    discriminator_optim.apply_gradients(zip(disc_grads, DISCRIMINATOR.trainable_weights))
        
    # GENERATOR TRAIN
    with tf.GradientTape() as gen_tape:
        generated_img = GENERATOR(noise, training=True)
        
        fake_out = DISCRIMINATOR(generated_img, training=False)
        
        gen_loss = generator_loss(fake_out)
        
    gen_gradients = gen_tape.gradient(gen_loss, GENERATOR.trainable_variables)
    generator_optim.apply_gradients(zip(gen_gradients, GENERATOR.trainable_variables))

In [ ]:
for epoch in range(2000):
    for imgs in tqdm(train_ds, desc=f'Epoch {epoch}', ascii=True):
        train_step(imgs)

In [ ]:
seed = tf.random.normal([N_IMG_TO_GENERATE, LATENT_DIM])

In [ ]:
generated_image = GENERATOR.predict(seed)

for i in range(generated_image.shape[0]):
    plt.subplot(2, 4, i+1)
    plt.imshow(generated_image[i], cmap='gray')
    plt.axis('off')